In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
df = pd.read_csv('../Artworks.csv', sep=',', header=0)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138210 entries, 0 to 138209
Data columns (total 29 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Title               138171 non-null  object 
 1   Artist              136895 non-null  object 
 2   ConstituentID       136895 non-null  object 
 3   ArtistBio           132146 non-null  object 
 4   Nationality         136895 non-null  object 
 5   BeginDate           136895 non-null  object 
 6   EndDate             136895 non-null  object 
 7   Gender              136895 non-null  object 
 8   Date                135983 non-null  object 
 9   Medium              127960 non-null  object 
 10  Dimensions          128080 non-null  object 
 11  CreditLine          135659 non-null  object 
 12  AccessionNumber     138210 non-null  object 
 13  Classification      138210 non-null  object 
 14  Department          138210 non-null  object 
 15  DateAcquired        131067 non-nul

In [5]:
df.drop('Seat Height (cm)', axis=1, inplace=True)

In [6]:
df.shape

(138210, 28)

In [7]:
dfc = df

In [8]:
dfc[dfc['ConstituentID'].str.len() > 4].head(2)

,Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,Medium,...,URL,ThumbnailURL,Circumference (cm),Depth (cm),Diameter (cm),Height (cm),Length (cm),Weight (kg),Width (cm),Duration (sec.)
65,"House IV Project, Falls Village, Connecticut (...","Peter Eisenman, Robert Cole","6969, 8134","(American, born 1932)",(American) (),(1932) (0),(0) (0),(Male) (Male),1975,Ink and color ink on frosted polymer sheet,...,http://www.moma.org/collection/works/81,http://www.moma.org/media/W1siZiIsIjIxMDA0MSJd...,NaN,NaN,NaN,34.9251,NaN,NaN,113.3477,NaN
66,"Villa dall'Ava, Paris (Saint-Cloud), France, E...","Rem Koolhaas, Madelon Vriesendorp","6956, 6957","(Dutch, born 1944) (Dutch, born 1945)",(Dutch) (Dutch),(1944) (1945),(0) (0),(Male) (Female),1987,Synthetic polymer paint and ink on paper,...,http://www.moma.org/collection/works/82,http://www.moma.org/media/W1siZiIsIjYwMTEyIl0s...,NaN,NaN,NaN,63.5001,NaN,NaN,99.0602,NaN


In [9]:
dfc.iloc[66,:]

Title                 Villa dall'Ava, Paris (Saint-Cloud), France, E...
Artist                                Rem Koolhaas, Madelon Vriesendorp
ConstituentID                                                6956, 6957
ArtistBio                         (Dutch, born 1944) (Dutch, born 1945)
Nationality                                             (Dutch) (Dutch)
BeginDate                                                 (1944) (1945)
EndDate                                                         (0) (0)
Gender                                                  (Male) (Female)
Date                                                               1987
Medium                         Synthetic polymer paint and ink on paper
Dimensions                                    25 x 39" (63.5 x 99.1 cm)
CreditLine                                        Gift of Max Underwood
AccessionNumber                                                 21.2000
Classification                                             Archi

In [10]:
def explode(df, lst_cols, fill_value='', preserve_index=False):
    
    # make sure `lst_cols` is list-alike
    if (lst_cols is not None
        and len(lst_cols) > 0
        and not isinstance(lst_cols, (list, tuple, np.ndarray, pd.Series))):
        lst_cols = [lst_cols]
        
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)
    
    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()
    
    # preserve original index values    
    idx = np.repeat(df.index.values, lens)
    # create "exploded" DF
    res = (pd.DataFrame({
                col:np.repeat(df[col].values, lens)
                for col in idx_cols},
                index=idx)
             .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
                            for col in lst_cols}))
    
    # append those rows that have empty lists
    if (lens == 0).any():
        # at least one list in cells is empty
        res = (res.append(df.loc[lens==0, idx_cols], sort=False)
                  .fillna(fill_value))
        
    # revert the original index order
    res = res.sort_index()
    
    # reset index if requested
    if not preserve_index:        
        res = res.reset_index(drop=True)
    return res

In [11]:
dfc = dfc.astype(str)

**This is the column we want to separate**

ConstituentID                                                6956, 6957<br>

We will need ObjectID to reconnect each new row to the original dataset

In [12]:
x = dfc[['ObjectID','ConstituentID']]

In [13]:
objectid = explode(x.assign(CID=x.ConstituentID.str.split(',')), 'CID').ObjectID
constituentID = explode(x.assign(ConstID=x.ConstituentID.str.split(',')), 'ConstID').ConstID

In [14]:
result = pd.concat([objectid,constituentID], axis=1)

In [15]:
result.head(70)

,ObjectID,ConstID
0,2,6210
1,3,7470
2,4,7605
3,5,7056
4,6,7605
...,...,...
65,81,6969
66,81,8134
67,82,6956
68,82,6957


In [16]:
result.shape

(153917, 2)

In [17]:
result['ObjectID'] = result.ObjectID.astype(int)

In [18]:
#final_df = df.join(result, on='ObjectID', how='right', lsuffix='ObjectID', rsuffix='ObjectID')

In [19]:
final_df = pd.merge(result, df, how='inner', on='ObjectID')

In [20]:
final_df.head()

,ObjectID,ConstID,Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,...,URL,ThumbnailURL,Circumference (cm),Depth (cm),Diameter (cm),Height (cm),Length (cm),Weight (kg),Width (cm),Duration (sec.)
0,2,6210,"Ferdinandsbrücke Project, Vienna, Austria (Ele...",Otto Wagner,6210,"(Austrian, 1841–1918)",(Austrian),(1841),(1918),(Male),...,http://www.moma.org/collection/works/2,http://www.moma.org/media/W1siZiIsIjU5NDA1Il0s...,NaN,NaN,NaN,48.6000,NaN,NaN,168.9000,NaN
1,3,7470,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,7470,"(French, born 1944)",(French),(1944),(0),(Male),...,http://www.moma.org/collection/works/3,http://www.moma.org/media/W1siZiIsIjk3Il0sWyJw...,NaN,NaN,NaN,40.6401,NaN,NaN,29.8451,NaN
2,4,7605,"Villa near Vienna Project, Outside Vienna, Aus...",Emil Hoppe,7605,"(Austrian, 1876–1957)",(Austrian),(1876),(1957),(Male),...,http://www.moma.org/collection/works/4,http://www.moma.org/media/W1siZiIsIjk4Il0sWyJw...,NaN,NaN,NaN,34.3000,NaN,NaN,31.8000,NaN
3,5,7056,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,"(French and Swiss, born Switzerland 1944)",(),(1944),(0),(Male),...,http://www.moma.org/collection/works/5,http://www.moma.org/media/W1siZiIsIjEyNCJdLFsi...,NaN,NaN,NaN,50.8000,NaN,NaN,50.8000,NaN
4,6,7605,"Villa, project, outside Vienna, Austria, Exter...",Emil Hoppe,7605,"(Austrian, 1876–1957)",(Austrian),(1876),(1957),(Male),...,http://www.moma.org/collection/works/6,http://www.moma.org/media/W1siZiIsIjEyNiJdLFsi...,NaN,NaN,NaN,38.4000,NaN,NaN,19.1000,NaN


In [21]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 153917 entries, 0 to 153916
Data columns (total 29 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ObjectID            153917 non-null  int64  
 1   ConstID             153917 non-null  object 
 2   Title               153876 non-null  object 
 3   Artist              152602 non-null  object 
 4   ConstituentID       152602 non-null  object 
 5   ArtistBio           147685 non-null  object 
 6   Nationality         152602 non-null  object 
 7   BeginDate           152602 non-null  object 
 8   EndDate             152602 non-null  object 
 9   Gender              152602 non-null  object 
 10  Date                151338 non-null  object 
 11  Medium              142850 non-null  object 
 12  Dimensions          142634 non-null  object 
 13  CreditLine          151143 non-null  object 
 14  AccessionNumber     153917 non-null  object 
 15  Classification      153917 non-nul

In [22]:
final_df.columns = ['ObjectID', 'ArtistID', 'Title', 'All_Artists', 'All_IDs',
       'All_Bios', 'All_Nationalities', 'All_BeginDates', 'All_EndDates', 'All_Genders', 'Date',
       'Medium', 'Dimensions', 'CreditLine', 'AccessionNumber',
       'Classification', 'Department', 'DateAcquired', 'Cataloged', 'URL', 'ThumbnailURL', 'Circumference (cm)',
       'Depth (cm)', 'Diameter (cm)', 'Height (cm)', 'Length (cm)',
       'Weight (kg)', 'Width (cm)', 'Duration (sec.)']

In [23]:
len(final_df.columns)

29

In [24]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 153917 entries, 0 to 153916
Data columns (total 29 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ObjectID            153917 non-null  int64  
 1   ArtistID            153917 non-null  object 
 2   Title               153876 non-null  object 
 3   All_Artists         152602 non-null  object 
 4   All_IDs             152602 non-null  object 
 5   All_Bios            147685 non-null  object 
 6   All_Nationalities   152602 non-null  object 
 7   All_BeginDates      152602 non-null  object 
 8   All_EndDates        152602 non-null  object 
 9   All_Genders         152602 non-null  object 
 10  Date                151338 non-null  object 
 11  Medium              142850 non-null  object 
 12  Dimensions          142634 non-null  object 
 13  CreditLine          151143 non-null  object 
 14  AccessionNumber     153917 non-null  object 
 15  Classification      153917 non-nul

In [25]:
final_df[final_df['All_IDs'].str.len() > 4].head(3)

,ObjectID,ArtistID,Title,All_Artists,All_IDs,All_Bios,All_Nationalities,All_BeginDates,All_EndDates,All_Genders,...,URL,ThumbnailURL,Circumference (cm),Depth (cm),Diameter (cm),Height (cm),Length (cm),Weight (kg),Width (cm),Duration (sec.)
65,81,6969,"House IV Project, Falls Village, Connecticut (...","Peter Eisenman, Robert Cole","6969, 8134","(American, born 1932)",(American) (),(1932) (0),(0) (0),(Male) (Male),...,http://www.moma.org/collection/works/81,http://www.moma.org/media/W1siZiIsIjIxMDA0MSJd...,NaN,NaN,NaN,34.9251,NaN,NaN,113.3477,NaN
66,81,8134,"House IV Project, Falls Village, Connecticut (...","Peter Eisenman, Robert Cole","6969, 8134","(American, born 1932)",(American) (),(1932) (0),(0) (0),(Male) (Male),...,http://www.moma.org/collection/works/81,http://www.moma.org/media/W1siZiIsIjIxMDA0MSJd...,NaN,NaN,NaN,34.9251,NaN,NaN,113.3477,NaN
67,82,6956,"Villa dall'Ava, Paris (Saint-Cloud), France, E...","Rem Koolhaas, Madelon Vriesendorp","6956, 6957","(Dutch, born 1944) (Dutch, born 1945)",(Dutch) (Dutch),(1944) (1945),(0) (0),(Male) (Female),...,http://www.moma.org/collection/works/82,http://www.moma.org/media/W1siZiIsIjYwMTEyIl0s...,NaN,NaN,NaN,63.5001,NaN,NaN,99.0602,NaN


In [26]:
final_df.head()

,ObjectID,ArtistID,Title,All_Artists,All_IDs,All_Bios,All_Nationalities,All_BeginDates,All_EndDates,All_Genders,...,URL,ThumbnailURL,Circumference (cm),Depth (cm),Diameter (cm),Height (cm),Length (cm),Weight (kg),Width (cm),Duration (sec.)
0,2,6210,"Ferdinandsbrücke Project, Vienna, Austria (Ele...",Otto Wagner,6210,"(Austrian, 1841–1918)",(Austrian),(1841),(1918),(Male),...,http://www.moma.org/collection/works/2,http://www.moma.org/media/W1siZiIsIjU5NDA1Il0s...,NaN,NaN,NaN,48.6000,NaN,NaN,168.9000,NaN
1,3,7470,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,7470,"(French, born 1944)",(French),(1944),(0),(Male),...,http://www.moma.org/collection/works/3,http://www.moma.org/media/W1siZiIsIjk3Il0sWyJw...,NaN,NaN,NaN,40.6401,NaN,NaN,29.8451,NaN
2,4,7605,"Villa near Vienna Project, Outside Vienna, Aus...",Emil Hoppe,7605,"(Austrian, 1876–1957)",(Austrian),(1876),(1957),(Male),...,http://www.moma.org/collection/works/4,http://www.moma.org/media/W1siZiIsIjk4Il0sWyJw...,NaN,NaN,NaN,34.3000,NaN,NaN,31.8000,NaN
3,5,7056,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,"(French and Swiss, born Switzerland 1944)",(),(1944),(0),(Male),...,http://www.moma.org/collection/works/5,http://www.moma.org/media/W1siZiIsIjEyNCJdLFsi...,NaN,NaN,NaN,50.8000,NaN,NaN,50.8000,NaN
4,6,7605,"Villa, project, outside Vienna, Austria, Exter...",Emil Hoppe,7605,"(Austrian, 1876–1957)",(Austrian),(1876),(1957),(Male),...,http://www.moma.org/collection/works/6,http://www.moma.org/media/W1siZiIsIjEyNiJdLFsi...,NaN,NaN,NaN,38.4000,NaN,NaN,19.1000,NaN


In [27]:
corrections = final_df.select_dtypes(include='object').apply(lambda x: x.str.replace(',',' /'))

In [28]:
corrections = corrections.apply(lambda x: x.str.replace('\n',' and'))

In [29]:
corrections = corrections.apply(lambda x: x.str.replace('\r',' and'))

In [30]:
corrections = corrections.apply(lambda x: x.str.replace('nan',''))

In [31]:
final_df.iloc[:, 1:21] = corrections

In [40]:
final_df.replace(0, np.nan, inplace=True)

In [41]:
final_df.to_csv('Artworks_Edits.csv',sep=',', index=False, encoding='UTF8')

In [250]:
#mask = np.column_stack([final_df[col].str.contains(r"\r", na=False) for col in final_df.select_dtypes('object')])
#final_df.loc[mask.any(axis=1)]

,ObjectID,ArtistID,Title,All_Artists,All_IDs,All_Bios,All_Nationalities,All_BeginDates,All_EndDates,All_Genders,...,URL,ThumbnailURL,Circumference (cm),Depth (cm),Diameter (cm),Height (cm),Length (cm),Weight (kg),Width (cm),Duration (sec.)
76,96,2308,French & Company / project / New York (Manhattan) / New York / Facade study,Michael Graves,2308,(American / 1934–2015),(American),(1934),(2015),(Male),...,http://www.moma.org/collection/works/96,http://www.moma.org/media/W1siZiIsIjY0MiJdLFsicCIsImNvbnZlcnQiLCItcmVzaXplIDMwMHgzMDBcdTAwM2UiXV0.jpg?sha=53611c85c177beee,NaN,NaN,NaN,27.60,NaN,NaN,27.3000,NaN
77,98,2308,Vacation House / project / Aspen / Colorado / Fireplace,Michael Graves,2308,(American / 1934–2015),(American),(1934),(2015),(Male),...,http://www.moma.org/collection/works/98,http://www.moma.org/media/W1siZiIsIjY1OSJdLFsicCIsImNvbnZlcnQiLCItcmVzaXplIDMwMHgzMDBcdTAwM2UiXV0.jpg?sha=42b1a231ce0d7234,NaN,NaN,NaN,20.30,NaN,NaN,20.3000,NaN
90,123,27,Skandia Cinema / Stockholm / Sweden / Plan of ground floor,Erik Gunnar Asplund,27,(Swedish / 1885–1940),(Swedish),(1885),(1940),(Male),...,http://www.moma.org/collection/works/123,http://www.moma.org/media/W1siZiIsIjkxNiJdLFsicCIsImNvbnZlcnQiLCItcmVzaXplIDMwMHgzMDBcdTAwM2UiXV0.jpg?sha=da4ea6eb2ff477c4,NaN,NaN,NaN,47.90,NaN,NaN,55.2000,NaN
138,174,8219,Hadassah University Medical Center / Mount Scopus / Jerusalem / Perspectives,Erich Mendelsohn,8219,(American / born Germany (now Poland). 1887–1953),(American),(1887),(1953),(Male),...,http://www.moma.org/collection/works/174,http://www.moma.org/media/W1siZiIsIjIxMDIyNyJdLFsicCIsImNvbnZlcnQiLCItcmVzaXplIDMwMHgzMDBcdTAwM2UiXV0.jpg?sha=957f2a1ca3763a44,NaN,NaN,NaN,21.59,NaN,NaN,13.6525,NaN
164,201,8707,Slow House Project / North Haven / New York / Plan of lower-level and sections,Diller + Scofidio / Elizabeth Diller / Ricardo Scofidio,8707 / 6951 / 6952,(American / established 1979) (American / born Poland 1954) (American / born 1935),(American) (American) (American),(1979) (1954) (1935),(0) (0) (0),() (Female) (Male),...,http://www.moma.org/collection/works/201,http://www.moma.org/media/W1siZiIsIjE5MjQiXSxbInAiLCJjb252ZXJ0IiwiLXJlc2l6ZSAzMDB4MzAwXHUwMDNlIl1d.jpg?sha=5aec1fba71220ffe,NaN,3.8,NaN,121.00,NaN,NaN,92.7000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153909,410389,131526,Anima espresso cup,Kosuke Araki,131526,(Japanese / born 1988),(Japanese),(1988),(0),(Female),...,NaN,NaN,NaN,6.4,NaN,6.00,NaN,NaN,8.5000,NaN
153910,410390,131526,Anima plate,Kosuke Araki,131526,(Japanese / born 1988),(Japanese),(1988),(0),(Female),...,NaN,NaN,NaN,12.5,NaN,2.00,NaN,NaN,12.5000,NaN
153911,410391,131526,Anima saucer/plate,Kosuke Araki,131526,(Japanese / born 1988),(Japanese),(1988),(0),(Female),...,NaN,NaN,NaN,11.8,NaN,1.50,NaN,NaN,11.8000,NaN
153912,410392,131526,Anima saucer/plate,Kosuke Araki,131526,(Japanese / born 1988),(Japanese),(1988),(0),(Female),...,NaN,NaN,NaN,13.5,NaN,1.50,NaN,NaN,13.5000,NaN


<br>
<br>
<br>

## Separate More Columns

**If we wanted to separate more columns like we did before, we could do it for the following ones, although this is not recommended becasue the other existing dataset (artists) already has the information**

Artist                                Rem Koolhaas, Madelon Vriesendorp<br>
ConstituentID                                                6956, 6957<br>
ArtistBio                         (Dutch, born 1944) (Dutch, born 1945)<br>
Nationality                                             (Dutch) (Dutch)<br>
BeginDate                                                 (1944) (1945)<br>
EndDate                                                         (0) (0)<br>
Gender                                                  (Male) (Female)<br>

In [ ]:
x = dfc[['ObjectID','Artist', 'ConstituentID','ArtistBio','Nationality','BeginDate','EndDate','Gender']]

In [333]:
objectid = explode(x.assign(CID=x.ConstituentID.str.split(',')), 'CID').ObjectID
artist = explode(x.assign(Artist=x.Artist.str.split(',')), 'Artist').Artist
constituentID = explode(x.assign(ConstID=x.ConstituentID.str.split(',')), 'ConstID').ConstID
artistbio = explode(x.assign(Artistbio=x.ArtistBio.str.split('\) \(')), 'Artistbio').Artistbio
nationality = explode(x.assign(nationality=x.Nationality.str.split(' ')), 'nationality').nationality
begindate = explode(x.assign(begindate=x.BeginDate.str.split(' ')), 'begindate').begindate
enddate = explode(x.assign(enddate=x.EndDate.str.split(' ')), 'enddate').enddate
gender = explode(x.assign(gender=x.Gender.str.split(' ')), 'gender').gender

In [334]:
result = pd.concat([objectid,artist,constituentID,artistbio,nationality,begindate,enddate,gender], axis=1)

In [335]:
result.head(70)

,ObjectID,Artist,ConstID,Artistbio,nationality,begindate,enddate,gender
0,2,Otto Wagner,6210,"(Austrian, 1841–1918)",(Austrian),(1841),(1918),(Male)
1,3,Christian de Portzamparc,7470,"(French, born 1944)",(French),(1944),(0),(Male)
2,4,Emil Hoppe,7605,"(Austrian, 1876–1957)",(Austrian),(1876),(1957),(Male)
3,5,Bernard Tschumi,7056,"(French and Swiss, born Switzerland 1944)",(),(1944),(0),(Male)
4,6,Emil Hoppe,7605,"(Austrian, 1876–1957)",(Austrian),(1876),(1957),(Male)
...,...,...,...,...,...,...,...,...
65,81,Peter Eisenman,6969,"(American, born 1932)",(American),(1932),(0),(Male)
66,81,Robert Cole,8134,"(Dutch, born 1944",(),(0),(0),(Male)
67,82,Rem Koolhaas,6956,"Dutch, born 1945)",(Dutch),(1944),(0),(Male)
68,82,Madelon Vriesendorp,6957,"(American, 1949–1991)",(Dutch),(1945),(0),(Female)


In [336]:
result.shape

(155771, 8)

In [337]:
result.dropna(subset=['ObjectID'], inplace=True)

In [338]:
result['ObjectID'] = result.ObjectID.astype(int)

In [339]:
final_df = pd.merge(df, result, how='inner', on='ObjectID')

In [340]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 153917 entries, 0 to 153916
Data columns (total 36 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ObjectIDObjectID    153917 non-null  int64  
 1   ArtistObjectID      153917 non-null  object 
 2   ConstID             153917 non-null  object 
 3   Artistbio           151364 non-null  object 
 4   nationality         153917 non-null  object 
 5   begindate           153917 non-null  object 
 6   enddate             153917 non-null  object 
 7   gender              153917 non-null  object 
 8   Title               109701 non-null  object 
 9   ArtistObjectID      109009 non-null  object 
 10  ConstituentID       109009 non-null  object 
 11  ArtistBio           105497 non-null  object 
 12  Nationality         109009 non-null  object 
 13  BeginDate           109009 non-null  object 
 14  EndDate             109009 non-null  object 
 15  Gender              109009 non-nul

In [341]:
final_df.drop(labels=['ArtistObjectID', 'ConstituentID', 'ArtistBio', 'Nationality', 'BeginDate', 'EndDate', 'Gender'], axis=1, inplace=True)

In [342]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 153917 entries, 0 to 153916
Data columns (total 28 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ObjectIDObjectID    153917 non-null  int64  
 1   ConstID             153917 non-null  object 
 2   Artistbio           151364 non-null  object 
 3   nationality         153917 non-null  object 
 4   begindate           153917 non-null  object 
 5   enddate             153917 non-null  object 
 6   gender              153917 non-null  object 
 7   Title               109701 non-null  object 
 8   Date                108466 non-null  object 
 9   Medium              100367 non-null  object 
 10  Dimensions          102524 non-null  object 
 11  CreditLine          107391 non-null  object 
 12  AccessionNumber     109718 non-null  object 
 13  Classification      109718 non-null  object 
 14  Department          109718 non-null  object 
 15  DateAcquired        106396 non-nul

In [343]:
final_df.to_csv('Data/Artists_Edited_2.csv',sep=',')